### Импортируем библиотеки

In [1]:
import pickle
import numpy as np
import pandas as pd
from docx import Document

### Считываем исходную таблицу из файла

In [2]:
def parse_docx_table(filename: str):
    document = Document(filename)

    tables = []
    for table in document.tables:
        # список списков с пустыми строками - структура будущего датафрейма
        df = [['' for x in range(len(table.columns))] for y in range(len(table.rows))]
        # заполняем дф данными из табл
        for x, row in enumerate(table.rows):
            for y, cell in enumerate(row.cells):
                if cell.text:
                    df[x][y] = cell.text
        tables.append(pd.DataFrame(df))

    return tables

In [3]:
init_df = parse_docx_table('DE_task_table.docx')[0] # берем первую таблицу из списка
header = init_df.iloc[0]                            # заголовк таблицы - первый ряд
init_df = init_df[1:]                               # убираем первый ряд из таблицы
init_df.columns = header                            # назначаем название колонок дф

In [4]:
# Переводим формат дат, год 9999 заменяем на nan обработкой ошибок
init_df['start_date']= pd.to_datetime(init_df['start_date'], format="%d.%m.%Y", errors = 'coerce',)
init_df['finish_date']= pd.to_datetime(init_df['finish_date'], format="%d.%m.%Y", errors = 'coerce',)

In [5]:
# Правило №2
init_df.loc[init_df['finish_date'].isnull(), 'finish_date'] = init_df['end_da']

/home/varsey-fabuza/PycharmProjects/dashboard-formation/env/lib/python3.10/site-packages/pandas/core/arrays/datetimes.py:339: UserWarning: Parsing '31.10.2020' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return cls._from_sequence_not_strict(scalars, dtype=dtype, copy=copy)
/home/varsey-fabuza/PycharmProjects/dashboard-formation/env/lib/python3.10/site-packages/pandas/core/arrays/datetimes.py:339: UserWarning: Parsing '30.09.2020' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return cls._from_sequence_not_strict(scalars, dtype=dtype, copy=copy)


In [6]:
init_df

,tab_num,start_date,finish_date,wday_type01,wday_type02,wday_type03,wday_type04,wday_type05,wplace_type,end_da
1,15123,2020-09-02,2020-10-31,0,0,0,0,0,0,31.10.2020
2,16234,2020-09-20,2020-10-30,0,0,1,1,0,2,
3,17345,2020-09-28,2020-10-25,1,0,0,0,0,2,
4,17345,2020-10-26,2020-12-31,1,1,1,1,1,1,
5,18456,2020-09-02,2020-09-30,2,2,2,2,2,3,30.09.2020
6,19567,2020-09-02,2020-12-31,3,3,3,3,3,4,


In [7]:
# Словарь для каждого номера ряда исходной таблицы с датой старта и окончания режима
start_date_dict = dict(zip(range(init_df.shape[0]), init_df.start_date.to_list()))
finish_date_dict = dict(zip(range(init_df.shape[0]), init_df.finish_date.to_list()))

In [8]:
dash = pd.DataFrame()
# Для каждого ряда - свой датафрейм по правилам №1 и №2
for num, tab_num in enumerate(init_df.tab_num.to_list()):
    interim = pd.DataFrame()
    interim['ymd_date'] = pd.date_range(start='2020-09-01', end='2020-12-31', freq='D')
    # дополнительные фичи для правила №8
    interim['weekday'] = interim['ymd_date'].dt.dayofweek
    interim['week'] = interim['ymd_date'].dt.isocalendar().week
    interim['week'] = interim['week'] - interim['week'].min()
    interim['halfweek'] = interim['week'] // 2
    interim['tab_num'] = tab_num
    interim['start_date'] = start_date_dict[num]
    interim['finish_date'] = finish_date_dict[num]
    dash = pd.concat([dash, interim])
dash['to_be_at_office'] = np.nan

In [9]:
repeated_tab_num = [x for x in init_df[init_df['tab_num'].duplicated()].tab_num.values]
repeated_tab_num

['17345']

In [10]:
def process_duplicated_tab_num(init_df: pd.DataFrame, repeated_tab_nums: list):
    '''' учитывает разные даты начала-конца для повторяющихся табельных номеров'''
    dash_merged = init_df.copy()
    for tab_num in repeated_tab_nums:
        dash_merged = pd.concat(
            [
                dash_merged[~dash_merged.tab_num.isin([tab_num])],
                pd.merge(
                    dash_merged.loc[(dash_merged['tab_num'] == tab_num)][['ymd_date', 'weekday', 'tab_num']],
                    dash_merged.loc[
                        (dash_merged['tab_num'] == tab_num) &
                        (dash_merged['ymd_date'] <= dash_merged['finish_date']) &
                        (dash_merged['ymd_date'] >= dash_merged['start_date'])
                    ],
                    how='left'
                ).drop_duplicates()
            ]
        )
        print(dash_merged.shape)
    return dash_merged

In [11]:
dash_merged = process_duplicated_tab_num(dash, repeated_tab_num)

(610, 8)


### Самопроверка

In [12]:
dash.shape, dash_merged.shape, 122*5

((732, 8), (610, 8), 610)

In [13]:
assert dash_merged.shape[0] == 122 * len(dash.tab_num.unique())

In [14]:
dash_merged.sample(5)

,ymd_date,weekday,week,halfweek,tab_num,start_date,finish_date,to_be_at_office
47,2020-10-18,6,6,3,19567,2020-09-02,2020-12-31,NaN
27,2020-09-28,0,4,2,17345,2020-09-28,2020-10-25,NaN
112,2020-12-22,1,16,8,18456,2020-09-02,2020-09-30,NaN
31,2020-10-02,4,4,2,19567,2020-09-02,2020-12-31,NaN
82,2020-11-22,6,11,5,17345,2020-10-26,2020-12-31,NaN


In [15]:
with open('dash_merged.pickle', 'wb') as handle:
    pickle.dump(dash_merged, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('init_df.pickle', 'wb') as handle:
    pickle.dump(init_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Обработку "вручную"

In [16]:
# Словарь для каждого номера ряда исходной таблицы с датой старта и окончания режима
start_date_dict = dict(zip(range(init_df.shape[0]), init_df.start_date.to_list()))
finish_date_dict = dict(zip(range(init_df.shape[0]), init_df.finish_date.to_list()))

In [17]:
init_df.tab_num.unique()[0]

'15123'

In [18]:
# Правило №6

In [19]:
# офисная работа wplace_type = 0
tab_num_index = 0
dash_merged.loc[
    # Правило №7
    (dash_merged['tab_num'] == init_df.tab_num.to_list()[tab_num_index]) &
    (dash_merged['ymd_date'] <= finish_date_dict[tab_num_index].to_datetime64()) &
    (dash_merged['ymd_date'] >= start_date_dict[tab_num_index].to_datetime64()) &
    (dash_merged['weekday'] != 5) &
    (dash_merged['weekday'] != 6),
    'to_be_at_office'
] = 1

In [20]:
# смотрим что получилось
dash_merged[dash_merged['tab_num'] == '15123']

,ymd_date,weekday,week,halfweek,tab_num,start_date,finish_date,to_be_at_office
0,2020-09-01,1,0,0,15123,2020-09-02,2020-10-31,NaN
1,2020-09-02,2,0,0,15123,2020-09-02,2020-10-31,1.0
2,2020-09-03,3,0,0,15123,2020-09-02,2020-10-31,1.0
3,2020-09-04,4,0,0,15123,2020-09-02,2020-10-31,1.0
4,2020-09-05,5,0,0,15123,2020-09-02,2020-10-31,NaN
...,...,...,...,...,...,...,...,...
117,2020-12-27,6,16,8,15123,2020-09-02,2020-10-31,NaN
118,2020-12-28,0,17,8,15123,2020-09-02,2020-10-31,NaN
119,2020-12-29,1,17,8,15123,2020-09-02,2020-10-31,NaN
120,2020-12-30,2,17,8,15123,2020-09-02,2020-10-31,NaN


In [21]:
# Правило №5
# Удаленная работа wplace_type = 1
print(init_df.tab_num.to_list()[3])

17345


In [22]:
tab_num_index = 3
dash_merged.loc[
    (dash_merged['tab_num'] == init_df.tab_num.to_list()[tab_num_index]) &
    (dash_merged['ymd_date'] <= finish_date_dict[tab_num_index].to_datetime64()) &
    (dash_merged['ymd_date'] >= start_date_dict[tab_num_index].to_datetime64()) &
    (dash_merged['weekday'] != 5) &
    (dash_merged['weekday'] != 6),
    'to_be_at_office'
] = 0

In [23]:
dash_merged[dash_merged['tab_num'] == '17345']

,ymd_date,weekday,week,halfweek,tab_num,start_date,finish_date,to_be_at_office
0,2020-09-01,1,<NA>,<NA>,17345,NaT,NaT,NaN
1,2020-09-02,2,<NA>,<NA>,17345,NaT,NaT,NaN
2,2020-09-03,3,<NA>,<NA>,17345,NaT,NaT,NaN
3,2020-09-04,4,<NA>,<NA>,17345,NaT,NaT,NaN
4,2020-09-05,5,<NA>,<NA>,17345,NaT,NaT,NaN
...,...,...,...,...,...,...,...,...
117,2020-12-27,6,16,8,17345,2020-10-26,2020-12-31,NaN
118,2020-12-28,0,17,8,17345,2020-10-26,2020-12-31,0.0
119,2020-12-29,1,17,8,17345,2020-10-26,2020-12-31,0.0
120,2020-12-30,2,17,8,17345,2020-10-26,2020-12-31,0.0


In [24]:
# Смешанная работа wplace_type = 2
print(init_df.tab_num.to_list()[2])

17345


In [25]:
tab_num_index = 2
dash_merged.loc[
    (dash_merged['tab_num'] == init_df.tab_num.to_list()[tab_num_index]) &
    (dash_merged['ymd_date'] <= finish_date_dict[tab_num_index].to_datetime64()) &
    (dash_merged['ymd_date'] >= start_date_dict[tab_num_index].to_datetime64()) &
    (dash_merged['weekday'] != 5) &
    (dash_merged['weekday'] != 6) &
    (dash_merged['weekday'] == 0),
    'to_be_at_office'
] = 0

In [26]:
tab_num_index = 2
dash_merged.loc[
    (
        (dash_merged['tab_num'] == init_df.tab_num.to_list()[tab_num_index]) &
        (dash_merged['ymd_date'] < finish_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['ymd_date'] > start_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['weekday'] != 5) &
        (dash_merged['weekday'] != 6)
    ) &
    (
        (dash_merged['weekday'].isin([1, 2, 3, 4]))
    ),
    'to_be_at_office'
] = 1

In [27]:
# Смешанная работа wplace_type = 2
print(init_df.tab_num.to_list()[1])

16234


In [28]:
tab_num_index = 1
dash_merged.loc[
    (
        (dash_merged['tab_num'] == init_df.tab_num.to_list()[tab_num_index]) &
        (dash_merged['ymd_date'] <= finish_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['ymd_date'] >= start_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['weekday'] != 5) &
        (dash_merged['weekday'] != 6)
    ) &
    (
        (dash_merged['weekday'].isin([0, 1, 4]))
    ),
    'to_be_at_office'
] = 1

In [29]:
tab_num_index = 1
dash_merged.loc[
    (
        (dash_merged['tab_num'] == init_df.tab_num.to_list()[tab_num_index]) &
        (dash_merged['ymd_date'] <= finish_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['ymd_date'] >= start_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['weekday'] != 5) &
        (dash_merged['weekday'] != 6)
    )
        &
    (
        (dash_merged['weekday'].isin([2, 3]))
    ),
    'to_be_at_office'
] = 0

In [30]:
dash_merged[dash_merged['tab_num'] == init_df.tab_num.to_list()[1]]

,ymd_date,weekday,week,halfweek,tab_num,start_date,finish_date,to_be_at_office
0,2020-09-01,1,0,0,16234,2020-09-20,2020-10-30,NaN
1,2020-09-02,2,0,0,16234,2020-09-20,2020-10-30,NaN
2,2020-09-03,3,0,0,16234,2020-09-20,2020-10-30,NaN
3,2020-09-04,4,0,0,16234,2020-09-20,2020-10-30,NaN
4,2020-09-05,5,0,0,16234,2020-09-20,2020-10-30,NaN
...,...,...,...,...,...,...,...,...
117,2020-12-27,6,16,8,16234,2020-09-20,2020-10-30,NaN
118,2020-12-28,0,17,8,16234,2020-09-20,2020-10-30,NaN
119,2020-12-29,1,17,8,16234,2020-09-20,2020-10-30,NaN
120,2020-12-30,2,17,8,16234,2020-09-20,2020-10-30,NaN


In [31]:
# Правило №8а

In [32]:
# Смешанная работа wplace_type = 3 неделя через неделю
print(init_df.tab_num.to_list()[4])

18456


In [33]:
tab_num_index = 4
dash_merged.loc[
    (
        (dash_merged['tab_num'] == init_df.tab_num.to_list()[tab_num_index]) &
        (dash_merged['ymd_date'] <= finish_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['ymd_date'] >= start_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['weekday'] != 5) &
        (dash_merged['weekday'] != 6)
    )
        &
    (
        (dash_merged['weekday'].isin([0, 1, 2, 3, 4]))
    )
            &
    (
        (dash_merged['week'] % 2 == 0)
    ),
    'to_be_at_office'
] = 0

In [34]:
tab_num_index = 4
dash_merged.loc[
    (
        (dash_merged['tab_num'] == init_df.tab_num.to_list()[tab_num_index]) &
        (dash_merged['ymd_date'] <= finish_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['ymd_date'] >= start_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['weekday'] != 5) &
        (dash_merged['weekday'] != 6)
    )
        &
    (
        (dash_merged['weekday'].isin([0, 1, 2, 3, 4]))
    )
            &
    (
        (dash_merged['week'] % 2 == 1)
    ),
    'to_be_at_office'
] = 1

In [35]:
dash_merged[dash_merged['tab_num'] == init_df.tab_num.to_list()[4]]

,ymd_date,weekday,week,halfweek,tab_num,start_date,finish_date,to_be_at_office
0,2020-09-01,1,0,0,18456,2020-09-02,2020-09-30,NaN
1,2020-09-02,2,0,0,18456,2020-09-02,2020-09-30,0.0
2,2020-09-03,3,0,0,18456,2020-09-02,2020-09-30,0.0
3,2020-09-04,4,0,0,18456,2020-09-02,2020-09-30,0.0
4,2020-09-05,5,0,0,18456,2020-09-02,2020-09-30,NaN
...,...,...,...,...,...,...,...,...
117,2020-12-27,6,16,8,18456,2020-09-02,2020-09-30,NaN
118,2020-12-28,0,17,8,18456,2020-09-02,2020-09-30,NaN
119,2020-12-29,1,17,8,18456,2020-09-02,2020-09-30,NaN
120,2020-12-30,2,17,8,18456,2020-09-02,2020-09-30,NaN


In [36]:
# Правило №8б

In [37]:
# Смешанная работа wplace_type = 3 неделя через две
print(init_df.tab_num.to_list()[5])

19567


In [38]:
tab_num_index = 5
dash_merged.loc[
    (
        (dash_merged['tab_num'] == init_df.tab_num.to_list()[tab_num_index]) &
        (dash_merged['ymd_date'] <= finish_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['ymd_date'] >= start_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['weekday'] != 5) &
        (dash_merged['weekday'] != 6)
    )
        &
    (
        (dash_merged['weekday'].isin([0, 1, 2, 3, 4]))
    )
            &
    (
        (dash_merged['halfweek'] % 2 == 0)
    ),
    'to_be_at_office'
] = 0

In [39]:
tab_num_index = 5
dash_merged.loc[
    (
        (dash_merged['tab_num'] == init_df.tab_num.to_list()[tab_num_index]) &
        (dash_merged['ymd_date'] <= finish_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['ymd_date'] >= start_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['weekday'] != 5) &
        (dash_merged['weekday'] != 6)
    )
        &
    (
        (dash_merged['weekday'].isin([0, 1, 2, 3, 4]))
    )
            &
    (
        (dash_merged['halfweek'] % 2 == 1)
    ),
    'to_be_at_office'
] = 1

In [40]:
dash_merged[dash_merged['tab_num'] == init_df.tab_num.to_list()[5]]

,ymd_date,weekday,week,halfweek,tab_num,start_date,finish_date,to_be_at_office
0,2020-09-01,1,0,0,19567,2020-09-02,2020-12-31,NaN
1,2020-09-02,2,0,0,19567,2020-09-02,2020-12-31,0.0
2,2020-09-03,3,0,0,19567,2020-09-02,2020-12-31,0.0
3,2020-09-04,4,0,0,19567,2020-09-02,2020-12-31,0.0
4,2020-09-05,5,0,0,19567,2020-09-02,2020-12-31,NaN
...,...,...,...,...,...,...,...,...
117,2020-12-27,6,16,8,19567,2020-09-02,2020-12-31,NaN
118,2020-12-28,0,17,8,19567,2020-09-02,2020-12-31,0.0
119,2020-12-29,1,17,8,19567,2020-09-02,2020-12-31,0.0
120,2020-12-30,2,17,8,19567,2020-09-02,2020-12-31,0.0


### Результат

In [41]:
result = dash_merged[['tab_num', 'ymd_date', 'to_be_at_office']].sort_values(by=['tab_num', 'ymd_date']).\
    reset_index(drop=True)

In [42]:
result['ymd_date'] = pd.to_datetime(result['ymd_date']).dt.strftime("%d.%m.%Y")

In [43]:
result

,tab_num,ymd_date,to_be_at_office
0,15123,01.09.2020,NaN
1,15123,02.09.2020,1.0
2,15123,03.09.2020,1.0
3,15123,04.09.2020,1.0
4,15123,05.09.2020,NaN
...,...,...,...
605,19567,27.12.2020,NaN
606,19567,28.12.2020,0.0
607,19567,29.12.2020,0.0
608,19567,30.12.2020,0.0


### Автоматическая обработка

In [44]:
import pickle
import pandas as pd

In [45]:
with open("dash_merged.pickle", "rb") as handle:
    dash_merged_from_file = pickle.load(handle)

with open("init_df.pickle", "rb") as handle:
    init_df_from_file = pickle.load(handle)

In [46]:
init_df_from_file.shape, dash_merged_from_file.shape

((6, 10), (610, 8))

In [47]:
from rule_parser import RuleParser


In [48]:
rp = RuleParser(data=dash_merged_from_file, init_table=init_df_from_file)

In [49]:
rp.data

,ymd_date,weekday,week,halfweek,tab_num,start_date,finish_date,to_be_at_office
0,2020-09-01,1,0,0,15123,2020-09-02,2020-10-31,NaN
1,2020-09-02,2,0,0,15123,2020-09-02,2020-10-31,NaN
2,2020-09-03,3,0,0,15123,2020-09-02,2020-10-31,NaN
3,2020-09-04,4,0,0,15123,2020-09-02,2020-10-31,NaN
4,2020-09-05,5,0,0,15123,2020-09-02,2020-10-31,NaN
...,...,...,...,...,...,...,...,...
117,2020-12-27,6,16,8,17345,2020-10-26,2020-12-31,NaN
118,2020-12-28,0,17,8,17345,2020-10-26,2020-12-31,NaN
119,2020-12-29,1,17,8,17345,2020-10-26,2020-12-31,NaN
120,2020-12-30,2,17,8,17345,2020-10-26,2020-12-31,NaN


In [50]:
rp.transfrom(tab_num_index=0)

wtype 0


,ymd_date,weekday,week,halfweek,tab_num,start_date,finish_date,to_be_at_office
0,2020-09-01,1,0,0,15123,2020-09-02,2020-10-31,NaN
1,2020-09-02,2,0,0,15123,2020-09-02,2020-10-31,1.0
2,2020-09-03,3,0,0,15123,2020-09-02,2020-10-31,1.0
3,2020-09-04,4,0,0,15123,2020-09-02,2020-10-31,1.0
4,2020-09-05,5,0,0,15123,2020-09-02,2020-10-31,NaN
...,...,...,...,...,...,...,...,...
117,2020-12-27,6,16,8,17345,2020-10-26,2020-12-31,NaN
118,2020-12-28,0,17,8,17345,2020-10-26,2020-12-31,NaN
119,2020-12-29,1,17,8,17345,2020-10-26,2020-12-31,NaN
120,2020-12-30,2,17,8,17345,2020-10-26,2020-12-31,NaN


In [51]:
rp.transfrom(tab_num_index=3)

wtype 1


,ymd_date,weekday,week,halfweek,tab_num,start_date,finish_date,to_be_at_office
0,2020-09-01,1,0,0,15123,2020-09-02,2020-10-31,NaN
1,2020-09-02,2,0,0,15123,2020-09-02,2020-10-31,1.0
2,2020-09-03,3,0,0,15123,2020-09-02,2020-10-31,1.0
3,2020-09-04,4,0,0,15123,2020-09-02,2020-10-31,1.0
4,2020-09-05,5,0,0,15123,2020-09-02,2020-10-31,NaN
...,...,...,...,...,...,...,...,...
117,2020-12-27,6,16,8,17345,2020-10-26,2020-12-31,NaN
118,2020-12-28,0,17,8,17345,2020-10-26,2020-12-31,0.0
119,2020-12-29,1,17,8,17345,2020-10-26,2020-12-31,0.0
120,2020-12-30,2,17,8,17345,2020-10-26,2020-12-31,0.0


In [52]:
rp.transfrom(tab_num_index=4)


wtype 3


,ymd_date,weekday,week,halfweek,tab_num,start_date,finish_date,to_be_at_office
0,2020-09-01,1,0,0,15123,2020-09-02,2020-10-31,NaN
1,2020-09-02,2,0,0,15123,2020-09-02,2020-10-31,1.0
2,2020-09-03,3,0,0,15123,2020-09-02,2020-10-31,1.0
3,2020-09-04,4,0,0,15123,2020-09-02,2020-10-31,1.0
4,2020-09-05,5,0,0,15123,2020-09-02,2020-10-31,NaN
...,...,...,...,...,...,...,...,...
117,2020-12-27,6,16,8,17345,2020-10-26,2020-12-31,NaN
118,2020-12-28,0,17,8,17345,2020-10-26,2020-12-31,0.0
119,2020-12-29,1,17,8,17345,2020-10-26,2020-12-31,0.0
120,2020-12-30,2,17,8,17345,2020-10-26,2020-12-31,0.0


In [53]:
rp.transfrom(tab_num_index=5)

wtype 4


,ymd_date,weekday,week,halfweek,tab_num,start_date,finish_date,to_be_at_office
0,2020-09-01,1,0,0,15123,2020-09-02,2020-10-31,NaN
1,2020-09-02,2,0,0,15123,2020-09-02,2020-10-31,1.0
2,2020-09-03,3,0,0,15123,2020-09-02,2020-10-31,1.0
3,2020-09-04,4,0,0,15123,2020-09-02,2020-10-31,1.0
4,2020-09-05,5,0,0,15123,2020-09-02,2020-10-31,NaN
...,...,...,...,...,...,...,...,...
117,2020-12-27,6,16,8,17345,2020-10-26,2020-12-31,NaN
118,2020-12-28,0,17,8,17345,2020-10-26,2020-12-31,0.0
119,2020-12-29,1,17,8,17345,2020-10-26,2020-12-31,0.0
120,2020-12-30,2,17,8,17345,2020-10-26,2020-12-31,0.0


In [54]:
rp.data[rp.data['tab_num'] == init_df_from_file.tab_num.to_list()[5]]

,ymd_date,weekday,week,halfweek,tab_num,start_date,finish_date,to_be_at_office
0,2020-09-01,1,0,0,19567,2020-09-02,2020-12-31,NaN
1,2020-09-02,2,0,0,19567,2020-09-02,2020-12-31,0.0
2,2020-09-03,3,0,0,19567,2020-09-02,2020-12-31,0.0
3,2020-09-04,4,0,0,19567,2020-09-02,2020-12-31,0.0
4,2020-09-05,5,0,0,19567,2020-09-02,2020-12-31,NaN
...,...,...,...,...,...,...,...,...
117,2020-12-27,6,16,8,19567,2020-09-02,2020-12-31,NaN
118,2020-12-28,0,17,8,19567,2020-09-02,2020-12-31,0.0
119,2020-12-29,1,17,8,19567,2020-09-02,2020-12-31,0.0
120,2020-12-30,2,17,8,19567,2020-09-02,2020-12-31,0.0


In [55]:
rp.transfrom(tab_num_index=1)

wtype 2


,ymd_date,weekday,week,halfweek,tab_num,start_date,finish_date,to_be_at_office
0,2020-09-01,1,0,0,15123,2020-09-02,2020-10-31,NaN
1,2020-09-02,2,0,0,15123,2020-09-02,2020-10-31,1.0
2,2020-09-03,3,0,0,15123,2020-09-02,2020-10-31,1.0
3,2020-09-04,4,0,0,15123,2020-09-02,2020-10-31,1.0
4,2020-09-05,5,0,0,15123,2020-09-02,2020-10-31,NaN
...,...,...,...,...,...,...,...,...
117,2020-12-27,6,16,8,17345,2020-10-26,2020-12-31,NaN
118,2020-12-28,0,17,8,17345,2020-10-26,2020-12-31,0.0
119,2020-12-29,1,17,8,17345,2020-10-26,2020-12-31,0.0
120,2020-12-30,2,17,8,17345,2020-10-26,2020-12-31,0.0


In [56]:
rp.data[rp.data['tab_num'] == init_df_from_file.tab_num.to_list()[1]]

,ymd_date,weekday,week,halfweek,tab_num,start_date,finish_date,to_be_at_office
0,2020-09-01,1,0,0,16234,2020-09-20,2020-10-30,NaN
1,2020-09-02,2,0,0,16234,2020-09-20,2020-10-30,NaN
2,2020-09-03,3,0,0,16234,2020-09-20,2020-10-30,NaN
3,2020-09-04,4,0,0,16234,2020-09-20,2020-10-30,NaN
4,2020-09-05,5,0,0,16234,2020-09-20,2020-10-30,NaN
...,...,...,...,...,...,...,...,...
117,2020-12-27,6,16,8,16234,2020-09-20,2020-10-30,NaN
118,2020-12-28,0,17,8,16234,2020-09-20,2020-10-30,NaN
119,2020-12-29,1,17,8,16234,2020-09-20,2020-10-30,NaN
120,2020-12-30,2,17,8,16234,2020-09-20,2020-10-30,NaN


In [57]:
rp.transfrom(tab_num_index=2)

wtype 2


,ymd_date,weekday,week,halfweek,tab_num,start_date,finish_date,to_be_at_office
0,2020-09-01,1,0,0,15123,2020-09-02,2020-10-31,NaN
1,2020-09-02,2,0,0,15123,2020-09-02,2020-10-31,1.0
2,2020-09-03,3,0,0,15123,2020-09-02,2020-10-31,1.0
3,2020-09-04,4,0,0,15123,2020-09-02,2020-10-31,1.0
4,2020-09-05,5,0,0,15123,2020-09-02,2020-10-31,NaN
...,...,...,...,...,...,...,...,...
117,2020-12-27,6,16,8,17345,2020-10-26,2020-12-31,NaN
118,2020-12-28,0,17,8,17345,2020-10-26,2020-12-31,0.0
119,2020-12-29,1,17,8,17345,2020-10-26,2020-12-31,0.0
120,2020-12-30,2,17,8,17345,2020-10-26,2020-12-31,0.0


### Автоматический пайплайн

In [58]:
auto_process = RuleParser(data=dash_merged_from_file, init_table=init_df_from_file)

In [59]:
for row_num in range(init_df_from_file.shape[0]):
    print(row_num)
    auto_process.transfrom(tab_num_index=row_num)

0
wtype 0
1
wtype 2
2
wtype 2
3
wtype 1
4
wtype 3
5
wtype 4


In [60]:
auto_result = rp.data[['tab_num', 'ymd_date', 'to_be_at_office']].sort_values(by=['tab_num', 'ymd_date'])\
    .reset_index(drop=True)

In [61]:
auto_result['ymd_date'] = pd.to_datetime(auto_result['ymd_date']).dt.strftime("%d.%m.%Y")

### Автоматический результат

In [62]:
auto_result

,tab_num,ymd_date,to_be_at_office
0,15123,01.09.2020,NaN
1,15123,02.09.2020,1.0
2,15123,03.09.2020,1.0
3,15123,04.09.2020,1.0
4,15123,05.09.2020,NaN
...,...,...,...
605,19567,27.12.2020,NaN
606,19567,28.12.2020,0.0
607,19567,29.12.2020,0.0
608,19567,30.12.2020,0.0


In [74]:
auto_result.to_csv('result.txt', index=False)

### Сравнение авторезультата и результата

In [63]:
auto_result['to_be_at_office'] - result['to_be_at_office']

0      NaN
1      0.0
2      0.0
3      0.0
4      NaN
      ... 
605    NaN
606    0.0
607    0.0
608    0.0
609    0.0
Name: to_be_at_office, Length: 610, dtype: float64